Done:
1. Store root vectors in Phi+ in a list called roots. Store all linear combinations of root vectors with coefficients of 1 in a list called sym_vec_combs
2. Calculate and store the Ualpha under which the possible root vector combinations are not stable.

To do:
8. Can we tell which vectors lie in the same orbit based on their root vector combination, rather than by calculating the B-orbits by brute force?
3. For each B-orbit, pull the coefficients on each root vector. Test if each coefficient is zero, and add a list of all coefficients of 0 to a list that stores the root vectors in the zero set. Do the same for coefficients that have only entries from T and none from the Ualpha
4. __Automate the discovery of algebraic relationships between the coordinates to give the polynomial defining equations in the zero set and the nonzero set, probably using an algorithm in elimination theory. "The basic idea of the implicitization problem is to convert the parametrization into definining equations for V" (Cox Little O'Shea pp. 133. N.t. they use V to refer to the polynomial zero set)__
5. __Find a way to tell when an "orbit" is infinite. It's probably when there's an algebraic dependence relation among the coordinates that involves a coefficient on one of the root vectors (in addition to or instead of the variables standing for an arbitrary scalar entry in T or Ualpha). Find a rigorous way of stating and defending this. Modality is probably the useful concept here. Since the orbits are varieties, is it possible to calculate the modality of the individual "orbits" to see if they turn out to be families of orbits? Then an orbit family would have modality 1, and a singleton orbit would have modality zero.__
6. __Automate the reverse inclusion calculations in Python. Is it possible that it actually isn't necessary to check the reverse inclusion? When would it happen that the B-orbit is in the intersection of the zero set and the nonzero set but not vice versa?__
7. __Find a way to tell when we have calculated all the B-orbits. Automate proof of nilradical exhaustion. Is there a way we can tell how many infinite "orbits" and how many singleton "orbits" there are before we calculate them? If so, we could just keep count and do a proof of exhaustion of n when we've got enough. It would also be sufficient if we *can* tell which vectors lie in the same orbit based on their root vector combination.__
9. Put in the option to display a B-orbit in equation form
10. Implement class structure
11. Add error handling

Instead of root vector combination tuples, it would be easier to have a single tuple with an indexed set of root vector coefficients like in B & H

In [1]:
import numpy as np
from itertools import combinations
from collections import OrderedDict

In [3]:
"""
Make matrices for the root vectors for the positive roots in type B (i.e., the root vectors in the nilradical).
Store the root vector matrices in a dictionary called rvecs where the keys are the eigenvector names given on page 131
of Erdmann and the values are the eigenvectors as numpy arrays
"""
def make_rvecs_Bv1(l):
    rvecs = {}

    M = np.zeros((2*l+1,2*l+1),dtype = int)
    # Add root vectors b_i for roots of the form epsilon_i to rvecs
    for i in np.arange(1,l+1,1):
        # b_ij
        M[i,0] = 1
        M[0,l+i] = -1
        hilo = "b" + str(i)
        rvecs.setdefault(hilo,M)
        M = np.zeros((2*l+1,2*l+1),dtype = int)
        #print(i)
        
    for i in np.arange(1,l,1):
        for j in np.arange(1,l+1,1):
            # p_ij
            if i<j:
                M[i,l+j] = 1
                M[j,l+i] = -1
                hilo = "p" + str(i) + str(j)
                rvecs.setdefault(hilo,M)
                M = np.zeros((2*l+1,2*l+1),dtype = int)
            # m_ij
            if i != j and i<j:
                M[i,j] = 1
                M[l+j,l+i] = -1
                hilo = "m" + str(i) + str(j)
                rvecs.setdefault(hilo,M)
                M = np.zeros((2*l+1,2*l+1),dtype = int)
    #print(i,j)
    return rvecs

rvecs = make_rvecs_Bv1(3)
print(len(rvecs))

9


In [24]:
"""
Make matrices for the root vectors for the positive roots in type B (i.e., the root vectors in the nilradical).
Store the root vector matrices in a dictionary called rvecs where the keys are the eigenvector names given on page 131
of Erdmann and the values are the eigenvectors as numpy arrays

"mat_type" is the eigenvector matrix form given on page 131 of Erdmann
"indices" is a list of the i,j indices
"matrix" is the eigenvector corresponding to the root of specified type and indices
"coords" are the coordinates of the positive nonzero matrix entry, to be used for the coordinate function
"""
def make_rvecs_B(l):
    rvecs = {}

    M = np.zeros((2*l+1,2*l+1),dtype = int)
    # Add root vectors b_i for roots of the form epsilon_i to rvecs
    for i in np.arange(1,l+1,1):
        # b_ij
        M[i,0] = 1
        M[0,l+i] = -1
        hilo = "b" + str(i)
        coef = np.zeros(l, dtype = int) # Array of length the number of roots in the base
        coef[i-1:l] = 1
        coef = tuple(coef)
        rvecs.setdefault(coef, {"mat_type":"b", "index":i, "matrix":M, "coords":[i,0]}) # index 1 \leq i \leq l
        M = np.zeros((2*l+1,2*l+1),dtype = int)
    for i in np.arange(1,l,1):
        for j in np.arange(1,l+1,1):
            # p_ij
            if i<j:
                M[i,l+j] = 1
                M[j,l+i] = -1
                hilo = "p" + str(i) + str(j)
                coef = np.zeros(l, dtype = int) # Array of length the number of roots in the base
                coef[i-1:j] = 1
                coef[j-1:l] = 2
                coef = tuple(coef)
                rvecs.setdefault(coef, {"mat_type":"p", "index":[i,j], "matrix":M, "coords":[i,l+j]}) # indices 1 \leq i < j \leq l
                M = np.zeros((2*l+1,2*l+1),dtype = int)
            # m_ij
            if i != j and i<j:
                M[i,j] = 1
                M[l+j,l+i] = -1
                hilo = "m" + str(i) + str(j)
                coef = np.zeros(l, dtype = int) # Array of length the number of roots in the base
                coef[i-1:j-1] = 1 
                coef = tuple(coef)
                rvecs.setdefault(coef, {"mat_type":"m", "index":[i,j], "matrix":M, "coords":[i,j]})
                M = np.zeros((2*l+1,2*l+1), dtype = int)
    return rvecs

rvecs = make_rvecs_B(3)
print(rvecs)

{(1, 1, 1): {'mat_type': 'b', 'index': 1, 'matrix': array([[ 0,  0,  0,  0, -1,  0,  0],
       [ 1,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0]]), 'coords': [1, 0]}, (0, 1, 1): {'mat_type': 'b', 'index': 2, 'matrix': array([[ 0,  0,  0,  0,  0, -1,  0],
       [ 0,  0,  0,  0,  0,  0,  0],
       [ 1,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0]]), 'coords': [2, 0]}, (0, 0, 1): {'mat_type': 'b', 'index': 3, 'matrix': array([[ 0,  0,  0,  0,  0,  0, -1],
       [ 0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0],
       [ 1,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0]]), 'coords': [3, 0]}, (1, 2,

In [ ]:
"""
COULD EVENTUALLY MAKE A TUPLE LIKE IN B & H IS THE KEY, BUT ONLY AFTER DETERMINING THE POSITIVE ROOTS AND MAKING THE ROOT
VECTORS. THEN WE NEED A STANDARD WAY TO FIX AN ORDERING OF THE POSITIVE ROOTS (OR DO SO ARBITRARILY, BUT THIS ISN'T SO GOOD
FOR USERS)
"""

In [3]:
# Take the type and rank of the Lie group
L = RootSystem(["B",3]).root_space()
roots=list(L.positive_roots())
print(roots)
#roots[0].to_vector()
roots = [root.to_vector() for root in roots]
roots

[alpha[1], alpha[2], alpha[3], alpha[1] + alpha[2], alpha[2] + 2*alpha[3], alpha[2] + alpha[3], alpha[1] + alpha[2] + 2*alpha[3], alpha[1] + alpha[2] + alpha[3], alpha[1] + 2*alpha[2] + 2*alpha[3]]


[(1, 0, 0),
 (0, 1, 0),
 (0, 0, 1),
 (1, 1, 0),
 (0, 1, 2),
 (0, 1, 1),
 (1, 1, 2),
 (1, 1, 1),
 (1, 2, 2)]

In [30]:
"""
Make a matrix for the torus in type B.

Arguments: 
    Gtype: string. String specifying the group type: "A", "B", "C", or "D"

Returns:
    T: Sage symbolic matrix. Maximal torus of G with respect to a fixed set of positive (SIMPLE?) roots
    Sage symbolic class 'sage.modules.vector_rational_dense.Vector_rational_dense'
    IT: Sage symbolic matrix. Matrix inverse of the maximal torus of G with respect to a fixed set of 
    positive (SIMPLE?) roots
    Sage symbolic class 'sage.modules.vector_rational_dense.Vector_rational_dense'
"""
# Make a matrix for the torus given group type and rank
# Input for Gtype may be A,B,C,D
# None of these are accurate except the one for type B
def make_T(Gtype,l):
    if Gtype == 'A':
        n = l + 1
        #syms=np.array([var("q"+str(i)) for i in range(n)])
        #T = np.diag(syms)
    elif Gtype == 'B':
        n = 2*l+1
        # Make a numpy array of the symbolic entries in T
        syms = np.array([var("q"+str(i)) for i in range(l)], dtype = object)
        # Make a numpy array of the inverses of the symbolic entries in T
        symsinv = np.array([var("q"+str(i))^(-1) for i in range(l)], dtype = object)
        # Place a 1 in the top left entry of the diagonal
        syms = np.insert(syms,0,1)
        syms = np.insert(syms,l+1,symsinv)
        T = matrix(np.diag(syms))
    elif Gtype == 'C':
        n = 2*l
        #syms=np.array([var("q"+str(i)) for i in range(n)])
        #T = np.diag(syms)
    elif Gtype == 'D':
        n = 2*l
        #syms=np.array([var("q"+str(i)) for i in range(n)])
        #T = np.diag(syms)
    IT = T.inverse()
    return T,IT

In [39]:
"""
Make matrices for each unipotent root group of the positive roots for type B
"""
def make_Ua(l):
    Ua = {}
    i = 0
    for key, value in rvecs.items():
        name = "t" + str(key).replace("(", "").replace(")", "").replace(" ", "").replace(",", "")
        name = var(name)
        # Make the numpy array a Sage matrix so that it can be properly 
        # exponentiated (equiv. of MatrixExp), then take the matrix exponential
        U = matrix(value["matrix"]*name)
        U = e^U
        Ua.setdefault(key,U)
    return Ua
U = make_Ua(3)

In [41]:
"""
Create a list storing all combinations we want to sum of root vectors as symbols, 
excluding the empty sum but including single root vectors
"""
roots = np.array(list(rvecs.keys()))
for t in range(len(roots)):
    roots[t] = list(roots[t])
m = len(roots)

sym_vec_combs = []
for i in range(0, m):
    sym_vec_combs += list(combinations(roots,i))
print(len(sym_vec_combs))

511


        # If an entry involves some expression of t, change it to 
        # another variable. This doesn't work and we also probably
        # don't want it
        for i in range(2*l+1):
            for j in range(2*l+1):
                if U[i,j] != 0 and U[i,j] != 1:
                    U[i,j] = name
                    Ua[key] = U

In [42]:
roots = list(rvecs.keys())
for t in range(len(roots)):
    roots[t] = list(roots[t])
print(roots)
#print(type(x[0]))

[[1, 1, 1], [0, 1, 1], [0, 0, 1], [1, 2, 2], [1, 0, 0], [1, 1, 2], [1, 1, 0], [0, 1, 2], [0, 1, 0]]


In [46]:
# Take the type and rank of the Lie group
L = RootSystem(["B",3]).root_space()
roots=list(L.positive_roots())
print(roots)
#roots[0].to_vector()
roots = [root.to_vector() for root in roots]
roots

[alpha[1], alpha[2], alpha[3], alpha[1] + alpha[2], alpha[2] + 2*alpha[3], alpha[2] + alpha[3], alpha[1] + alpha[2] + 2*alpha[3], alpha[1] + alpha[2] + alpha[3], alpha[1] + 2*alpha[2] + 2*alpha[3]]


[(1, 0, 0),
 (0, 1, 0),
 (0, 0, 1),
 (1, 1, 0),
 (0, 1, 2),
 (0, 1, 1),
 (1, 1, 2),
 (1, 1, 1),
 (1, 2, 2)]

In [47]:
"""
Create a list of all combinations we want to sum of root vectors as symbols, 
excluding the empty sum but including single root vectors
"""
x_list=roots
sym_vec_combs = []
for i in range(1, len(x_list) + 1):
    sym_vec_combs += list(combinations(x_list,i))
print(len(sym_vec_combs))

511


In [48]:
"""
Fix a root x in Phi+, then loop through each root xi in Phi+ to check
whether x+xi is in Phi+. If x+xi is in Phi+, then x is not stable 
under Ui and we place Ui in a list with the same indexing as roots.

Index zero corresponds to alpha1, etc.
"""
Ualpha_list=[ [] for _ in range(len(roots)) ]
N = range(len(roots))
for f in N:
    for t in N:
        if roots[f]+roots[t] in roots:
            Ualpha_list[f].append(roots[t])

print(Ualpha_list)

[[(0, 1, 0), (0, 1, 2), (0, 1, 1)], [(1, 0, 0), (0, 0, 1), (1, 1, 2)], [(0, 1, 0), (1, 1, 0), (0, 1, 1), (1, 1, 1)], [(0, 0, 1), (0, 1, 2)], [(1, 0, 0), (1, 1, 0)], [(1, 0, 0), (0, 0, 1), (1, 1, 1)], [(0, 1, 0)], [(0, 0, 1), (0, 1, 1)], []]


In [49]:
tuple(Ualpha_list[0][0])

(0, 1, 0)

In [50]:
"""
Umat is basically Ualpha_list but with matrices
"""
Umat = []

for i in range(len(Ualpha_list)):
    dum = []
    for j in range(len(Ualpha_list[i])):
        #print(Ualpha_list[i][j])
        #print(type(U[tuple(Ualpha_list[0][0])]))
        dum.append(np.array(U[tuple(Ualpha_list[i][j])]))
    Umat.append(dum)

In [51]:
Umat[0]

[array([[1, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0],
        [0, 0, 1, t010, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, -t010, 1]], dtype=object),
 array([[1, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, t012],
        [0, 0, 0, 1, 0, -t012, 0],
        [0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 0, 1]], dtype=object),
 array([[1, 0, 0, 0, 0, -t011, 0],
        [0, 1, 0, 0, 0, 0, 0],
        [t011, 0, 1, 0, 0, -1/2*t011^2, 0],
        [0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 0, 1]], dtype=object)]

In [16]:
print(type(np.array(sym_vec_combs[10])))

<class 'numpy.ndarray'>


In [52]:
Ualpha_list

[[(0, 1, 0), (0, 1, 2), (0, 1, 1)],
 [(1, 0, 0), (0, 0, 1), (1, 1, 2)],
 [(0, 1, 0), (1, 1, 0), (0, 1, 1), (1, 1, 1)],
 [(0, 0, 1), (0, 1, 2)],
 [(1, 0, 0), (1, 1, 0)],
 [(1, 0, 0), (0, 0, 1), (1, 1, 1)],
 [(0, 1, 0)],
 [(0, 0, 1), (0, 1, 1)],
 []]

In [53]:
"""
Create a list xUlist to store the Ualpha under which the possible 
root vector combinations (stored in sym_vec_combs) are not stable.
Shares index with sym_vec_combs.
"""
xUlist = [ [] for _ in range(len(sym_vec_combs)) ]
xUlist_mat = [ [] for _ in range(len(sym_vec_combs)) ]
for k in range(len(sym_vec_combs)):
    for i in range(len(Ualpha_list)):
        if roots[i] in sym_vec_combs[k]:
            xUlist[k].extend(Ualpha_list[i])
            # xUlist_mat to store the Ualpha as matrices under which the possible 
            # root vector combinations (stored in sym_vec_combs) are not stable
            xUlist_mat[k].extend(Umat[i])

In [54]:
sym_vec_combs[33]

((1, 1, 0), (1, 1, 1))

In [55]:
"""
THIS WOULD BE BETTER AS A DICTIONARY

Save the products of U_alpha as matrices in a list (This is important)
THESE DIFFER FROM THE ONES CALCULATED IN B_OrbitAutomation_v3.1, MAYBE BECAUSE THE MATRICES ARE BEING MULTIPLIED IN
DIFFERENT ORDERS. WHY DOESN'T ORDER MATTER FOR THE PURPOSE OF CALCULATING B-ORBITS?
"""

Uprod = []
l = 7

for i in range(len(xUlist_mat)):
    dum = np.eye(l, dtype = int)
    for j in range(len(xUlist_mat[i])):
        new_mat = xUlist_mat[i][j]
        dum = np.matmul(dum,new_mat)
    Uprod.append(dum)
Uprod[1]

array([[1, 0, 0, 0, 0, 0, -t001],
       [0, 1, t100, 0, 0, 0, t112],
       [0, 0, 1, 0, 0, 0, 0],
       [t001, 0, 0, 1, -t112, 0, -1/2*t001^2],
       [0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, -t100, 1, 0],
       [0, 0, 0, 0, 0, 0, 1]], dtype=object)

In [21]:
Uprod[2]

array([[1, 0, 0, 0, -t111, -t011, 0],
       [t111, 1, 0, t110, -1/2*t111^2, 0, 0],
       [t011, 0, 1, t010, -t011*t111, -1/2*t011^2, 0],
       [0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, -t110, -t010, 1]], dtype=object)

In [23]:
rvecs[tuple(sym_vec_combs[0][0])]

array([[ 0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  1,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0, -1,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0]])

In [60]:
"""
THIS WOULD BE BETTER AS A DICTIONARY

Save the sums of the root vector combinations

using the matrices stored as values in the rvecs dictionary

PUT COEFFICIENTS ON THE ROOT VECTORS TO CHECK FOR INFINITE ORBIT FAMILIES
"""
l = 7
num = len(sym_vec_combs)
vsums = []

for i in range(num):
    dum = np.zeros(l, dtype = object)
    for j in range(len(sym_vec_combs[i])):
        value = rvecs[tuple(sym_vec_combs[i][j])]
        new_vec = value["matrix"]
        dum = dum + new_vec #np.matrix.sum(dum,new_vec)
    vsums.append(dum)
vsums[110]

array([[0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, -1, -1, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, -1, 0, 0]], dtype=object)

In [63]:
T, IT = make_T('B',3)

In [65]:
matrix(Uprod[0]).inverse()

[                      1                       0                       0                       0                       0                    t011                       0]
[                      0                       1                       0                       0                       0                       0                       0]
[                  -t011                       0                       1                   -t010                       0 -1/2*t011^2 - t010*t012                   -t012]
[                      0                       0                       0                       1                       0                    t012                       0]
[                      0                       0                       0                       0                       1                       0                       0]
[                      0                       0                       0                       0                       0                       1      

In [66]:
"""
Create a list of all possible B-orbits (potentially with repeats)
"""
B_list = []

for i in range(len(vsums)):
    B = T @ Uprod[i] @ vsums[i] @ matrix(Uprod[i]).inverse() @ IT
    B = simplify(B)
    B_list.append(B)
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [67]:
B_list[0]

array([[0, 0, 0, 0, q0*t011, 0, 0],
       [-q0*t011, 0, q0/q1, -q0*t010/q2, 0,
        -1/2*(t011^2 + 2*t010*t012)*q0*q1, -q0*q2*t012],
       [0, 0, 0, 0, 1/2*(t011^2 + 2*t010*t012)*q0*q1, 0, 0],
       [0, 0, 0, 0, q0*q2*t012, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, -q0/q1, 0, 0],
       [0, 0, 0, 0, q0*t010/q2, 0, 0]], dtype=object)

In [34]:
x = [tuple(root) for root in roots]
type(x[0])

<class 'tuple'>

In [ ]:
"""
Function that takes a B-orbit matrix as input and outputs the equation form
for G of type B
"""

def mat_to_eq(B_mat, sym_vec_combs, l):
    # Dictionary with keys the root vector combinations as tuples and values a dictionary of the positive roots and their 
    # corresponding coefficients, smth like {(0,1,0):{X112:qt100}}
    num = len(sym_vec_combs)
    # Make a list of positive roots that is an immutable object to use as key in dict
    roots = [tuple(root) for root in roots]
    eq = {}
    for i in range(num):
        for j in range(2*l+1):
            coef = {roots[j]:}
            eq.setdefault(sym_vec_combs[i],)
    eq = coef_x100*x100s+coef_x010*x010s+coef_x001*x001s+coef_x110*x110s\
    +coef_x011*x011s+coef_x012*x012s+coef_x111*x111s+coef_x112*x112s+coef_x122*x122s
    return eq #return [dictionary]

In [68]:
rvecs

{(1, 1, 1): {'mat_type': 'b',
  'index': 1,
  'matrix': array([[ 0,  0,  0,  0, -1,  0,  0],
         [ 1,  0,  0,  0,  0,  0,  0],
         [ 0,  0,  0,  0,  0,  0,  0],
         [ 0,  0,  0,  0,  0,  0,  0],
         [ 0,  0,  0,  0,  0,  0,  0],
         [ 0,  0,  0,  0,  0,  0,  0],
         [ 0,  0,  0,  0,  0,  0,  0]]),
  'coords': [1, 0]},
 (0, 1, 1): {'mat_type': 'b',
  'index': 2,
  'matrix': array([[ 0,  0,  0,  0,  0, -1,  0],
         [ 0,  0,  0,  0,  0,  0,  0],
         [ 1,  0,  0,  0,  0,  0,  0],
         [ 0,  0,  0,  0,  0,  0,  0],
         [ 0,  0,  0,  0,  0,  0,  0],
         [ 0,  0,  0,  0,  0,  0,  0],
         [ 0,  0,  0,  0,  0,  0,  0]]),
  'coords': [2, 0]},
 (0, 0, 1): {'mat_type': 'b',
  'index': 3,
  'matrix': array([[ 0,  0,  0,  0,  0,  0, -1],
         [ 0,  0,  0,  0,  0,  0,  0],
         [ 0,  0,  0,  0,  0,  0,  0],
         [ 1,  0,  0,  0,  0,  0,  0],
         [ 0,  0,  0,  0,  0,  0,  0],
         [ 0,  0,  0,  0,  0,  0,  0],
         [ 0

In [167]:
jk = tuple(tuple(x) for x in sym_vec_combs[i])


In [168]:
type(jk[0])

<class 'tuple'>

In [225]:
"""
For each root vector combination, store the 
"""

def coord_func(B_mat, sym_vec_combs, l):
    num = len(sym_vec_combs)
    
    coord_pulls = {}
    for i in range(num):
        vec_comb = tuple(tuple(x) for x in sym_vec_combs[i])
        coefs = {}
        for key, value in rvecs.items():
            try:
                hilo = str(value["mat_type"]) + str(value["index"][0]) + str(value["index"][1])
            except:
                hilo = str(value["mat_type"]) + str(value["index"])# + str(value["index"][1])
            row = value["coords"][0]
            col = value["coords"][1]
            #print(B_mat[i][row,col])
            coefs.setdefault(hilo,B_mat[i][row,col])
        coord_pulls.setdefault(vec_comb,coefs)
    return coord_pulls

In [252]:
def get_single_coefs(Bx):
    VZ_set = {}
    exprs = {}
    for key,value in Bx.items():
        z = []
        v = []
        expr = {}
        for vkey, vvalue in Bx[key].items():
            if vvalue == 0:
                z.append(vkey)
            elif "t" not in str(vvalue):
                v.append(vkey)
            else:
                expr.setdefault(vkey,vvalue)
        exprs.setdefault(key,expr)
        VZ_set.setdefault(key,{"nonzero_coefs":v,"zero_coefs":z})
    return VZ_set,exprs

In [268]:
def get_coefs(Bx):
    VZ_set = {}
    exprs = {}
    for key,value in Bx.items():
        z = []
        v = []
        expr = {}
        for vkey, vvalue in Bx[key].items():
                expr.setdefault(vkey,vvalue)
        exprs.setdefault(key,expr)
    return exprs

In [270]:
Bx = coord_func(B_list, sym_vec_combs, 3)
okay = get_coefs(Bx)
okay

{((1, 0, 0),): {'b1': -q0*t011,
  'b2': 0,
  'b3': 0,
  'p12': -1/2*(t011^2 + 2*t010*t012)*q0*q1,
  'm12': q0/q1,
  'p13': -q0*q2*t012,
  'm13': -q0*t010/q2,
  'p23': 0,
  'm23': 0},
 ((0, 1, 0),): {'b1': -q0*t001*t100,
  'b2': -q1*t001,
  'b3': 0,
  'p12': -q0*q1*t112,
  'm12': 0,
  'p13': -1/2*q0*q2*t001^2*t100,
  'm13': q0*t100/q2,
  'p23': -1/2*q1*q2*t001^2,
  'm23': q1/q2},
 ((0, 0, 1),): {'b1': q0*t110,
  'b2': q1*t010,
  'b3': q2,
  'p12': (q0*t011*t110 - q0*t010*t111)*q1,
  'm12': 0,
  'p13': -q0*q2*t111,
  'm13': 0,
  'p23': -q1*q2*t011,
  'm23': 0},
 ((1, 1, 0),): {'b1': -q0*t001,
  'b2': 0,
  'b3': 0,
  'p12': q0*q1*t012,
  'm12': 0,
  'p13': -1/2*q0*q2*t001^2,
  'm13': q0/q2,
  'p23': 0,
  'm23': 0},
 ((0, 1, 2),): {'b1': 0,
  'b2': 0,
  'b3': 0,
  'p12': -q0*q1*t110,
  'm12': 0,
  'p13': q0*q2*t100,
  'm13': 0,
  'p23': q1*q2,
  'm23': 0},
 ((0, 1, 1),): {'b1': q0*t100,
  'b2': q1,
  'b3': 0,
  'p12': -q0*q1*t111,
  'm12': 0,
  'p13': q0*q2*t001*t100,
  'm13': 0,
  'p23': 

In [253]:
Bx = coord_func(B_list, sym_vec_combs, 3)
VZ_set,exprs = get_single_coefs(Bx)

In [260]:
exprs

{((1, 0, 0),): {'b1': -q0*t011,
  'p12': -1/2*(t011^2 + 2*t010*t012)*q0*q1,
  'p13': -q0*q2*t012,
  'm13': -q0*t010/q2},
 ((0, 1, 0),): {'b1': -q0*t001*t100,
  'b2': -q1*t001,
  'p12': -q0*q1*t112,
  'p13': -1/2*q0*q2*t001^2*t100,
  'm13': q0*t100/q2,
  'p23': -1/2*q1*q2*t001^2},
 ((0, 0, 1),): {'b1': q0*t110,
  'b2': q1*t010,
  'p12': (q0*t011*t110 - q0*t010*t111)*q1,
  'p13': -q0*q2*t111,
  'p23': -q1*q2*t011},
 ((1, 1, 0),): {'b1': -q0*t001, 'p12': q0*q1*t012, 'p13': -1/2*q0*q2*t001^2},
 ((0, 1, 2),): {'p12': -q0*q1*t110, 'p13': q0*q2*t100},
 ((0, 1, 1),): {'b1': q0*t100,
  'p12': -q0*q1*t111,
  'p13': q0*q2*t001*t100,
  'p23': q1*q2*t001},
 ((1, 1, 2),): {'p12': q0*q1*t010},
 ((1, 1, 1),): {'p12': q0*q1*t011, 'p13': q0*q2*t001},
 ((1, 2, 2),): {},
 ((1, 0, 0), (0, 1, 0)): {'b1': -q0*t001*t100 - q0*t011,
  'b2': -q1*t001,
  'p12': -1/2*((t001^2*t010 + 2*t001*t011 - 2*t012)*q0*t100 - (2*(t001^2*t010 - (t001*t010 + t011)*t001 + t001*t011 - t012)*t010 + 2*(t001*t011 - t012)*t010 - 2*(t

In [ ]:
"""
What happens if we compute the Groebner bases and then check which of the expressions in the basis evaluates to something
that is certainly zero or nonzero?
What happens if we compute them in a way s.t. we can specify that q0,q1,q2!=0?
I should go through and check if these things give us the orbits we expect then come back for a way to justify them
"""

In [267]:
#SR.<b1,p12,p13,m13,t011,t010,t012,q0,q1,q2> = PolynomialRing(QQ,10)
b1,p12,p13,m13,t011,t010,t012,q0,q1,q2 = QQ['b1,p12,p13,m13,t011,t010,t012,q0,q1,q2'].gens()
I = ideal(-q0*t011, -1/2*(t011^2 + 2*t010*t012)*q0*q1, -q0*q2*t012, -q0*t010/q2)
B = I.groebner_basis()

AttributeError: 'Ideal_pid' object has no attribute 'groebner_basis'

In [259]:
for key, value in exprs.items():
    for vkey, vvalue in exprs[key].items():
        vkey = var(vkey)
        I = ideal()

<class 'sage.symbolic.expression.Expression'>
<class 'sage.symbolic.expression.Expression'>
<class 'sage.symbolic.expression.Expression'>
<class 'sage.symbolic.expression.Expression'>
<class 'sage.symbolic.expression.Expression'>
<class 'sage.symbolic.expression.Expression'>
<class 'sage.symbolic.expression.Expression'>
<class 'sage.symbolic.expression.Expression'>
<class 'sage.symbolic.expression.Expression'>
<class 'sage.symbolic.expression.Expression'>
<class 'sage.symbolic.expression.Expression'>
<class 'sage.symbolic.expression.Expression'>
<class 'sage.symbolic.expression.Expression'>
<class 'sage.symbolic.expression.Expression'>
<class 'sage.symbolic.expression.Expression'>
<class 'sage.symbolic.expression.Expression'>
<class 'sage.symbolic.expression.Expression'>
<class 'sage.symbolic.expression.Expression'>
<class 'sage.symbolic.expression.Expression'>
<class 'sage.symbolic.expression.Expression'>
<class 'sage.symbolic.expression.Expression'>
<class 'sage.symbolic.expression.E

<class 'sage.symbolic.expression.Expression'>
<class 'sage.symbolic.expression.Expression'>
<class 'sage.symbolic.expression.Expression'>
<class 'sage.symbolic.expression.Expression'>
<class 'sage.symbolic.expression.Expression'>
<class 'sage.symbolic.expression.Expression'>
<class 'sage.symbolic.expression.Expression'>
<class 'sage.symbolic.expression.Expression'>
<class 'sage.symbolic.expression.Expression'>
<class 'sage.symbolic.expression.Expression'>
<class 'sage.symbolic.expression.Expression'>
<class 'sage.symbolic.expression.Expression'>
<class 'sage.symbolic.expression.Expression'>
<class 'sage.symbolic.expression.Expression'>
<class 'sage.symbolic.expression.Expression'>
<class 'sage.symbolic.expression.Expression'>
<class 'sage.symbolic.expression.Expression'>
<class 'sage.symbolic.expression.Expression'>
<class 'sage.symbolic.expression.Expression'>
<class 'sage.symbolic.expression.Expression'>
<class 'sage.symbolic.expression.Expression'>
<class 'sage.symbolic.expression.E